In [1]:
import os
import sys
from pyspark import SparkContext, SparkConf
import json
import itertools
import math
import time
from queue import deque

In [2]:
appName = 'assignment4'
master = 'local[*]'
conf = SparkConf().setAppName(appName).setMaster(master)#.set('spark.jars.packages','graphframes:graphframes:0.6.0-spark2.3-s_2.11')
# conf = conf.setAll([('spark.executor.memory', '8g'), ('spark.executor.cores', '3'), ('spark.cores.max', '3'), ('spark.driver.memory','8g')])
sc = SparkContext(conf=conf)
sc.setLogLevel("INFO")
sc

<SparkContext master=local[*] appName=assignment4>

In [3]:
def create_adjacency_list(edges, vertices):
    adjacency_list = {}
    for (x, y) in edges:
        if(x in adjacency_list):
            adjacency_list[x].add(y)
        else:
            adjacency_list[x] = set([y])

        if(y in adjacency_list):
            adjacency_list[y].add(x)
        else:
            adjacency_list[y] = set([x])
    
    for v in set(vertices) - set(adjacency_list.keys()):
        adjacency_list[v] = set([])
            
    return adjacency_list

class Node:
    def __init__(self, idx, level, number_of_shortest_paths):
        self.id = idx
        self.level = level
        self.parent = []
        self.children = []
        self.number_of_shortest_paths = number_of_shortest_paths
        self.credit = 1
        return
    
    def __str__(self):
        return "{}, {}, {}".format(self.id, self.level, self.number_of_shortest_paths)
    
    def __repr__(self):
        return str(self)
    
class LevelIDMapper:
    def __init__(self):
        self.map = {}
        
    def update(self, level, node):
        if(level in self.map):
            self.map[level][node.id] = node
        else:
            self.map[level] = {node.id: node}
            
    def check_level(self, level, node_id):
        if(not level in self.map):
            return False
        
        return node_id in self.map[level]
    
    def get_node(self, level, node_id):
        
        if(not level in self.map):
            return False
        
        if(not node_id in self.map[level]):
            return False
        
        return self.map[level][node_id]
    
    
def BFS(start, adjacency_list):
    q = deque()
    start_node = Node(start, 0, 1)
    q.append(start_node)
    visited = set()
    visited.add(start)
    leaf_nodes = []
    level_node_map = LevelIDMapper()
    level_node_map.update(0, start_node)
    
    while(len(q) != 0):
        
        current_node = q.popleft()
        
        for child in adjacency_list[current_node.id]:
            if(child in visited and level_node_map.check_level(current_node.level+1, child)):
                child_node = level_node_map.get_node(current_node.level+1, child)
                child_node.parent.append(current_node)
                current_node.children.append(child_node)
                child_node.number_of_shortest_paths += current_node.number_of_shortest_paths
                level_node_map.update(current_node.level+1, child_node)
            if(not child in visited):
                child_node = Node(child, current_node.level+1, current_node.number_of_shortest_paths)
                child_node.parent.append(current_node)
                current_node.children.append(child_node)
                visited.add(child_node.id)
                q.append(child_node)
                level_node_map.update(current_node.level+1, child_node)
    
    
    return level_node_map, tuple(sorted(visited))
            

class EdgeCredit:
    def __init__(self):
        self.map = {}
    
    def update(self, edge, credit):
        edge = tuple(sorted(edge))
        if(edge in self.map):
            self.map[edge] += credit
        else:
            self.map[edge] = credit
            
    def divide_by_2(self):
        for edge in self.map.keys():
            self.map[edge] = self.map[edge] / 2.0
        return

def calculate_modularity(adjacency_list, communities, m):
    communities = list(communities)
    def helper(community):
        modularity = 0
        for (i, j) in itertools.combinations(community, 2):
            Aij = 1 if j in adjacency_list[i] else 0
            ki = len(adjacency_list[i])
            kj = len(adjacency_list[j])
            modularity += Aij-((ki*kj)/(2*m))
            
        return modularity
    
    modularity = sc.parallelize(communities).map(helper).reduce(lambda x, y: x+y)
        
            
    return modularity/(2*m)
            
    
    
def calculate_betweeness_and_communities(adjacency_list, vertices):
    ec = EdgeCredit()
    communities = set()
    for v in vertices:
        level_node_map, community = BFS(v, adjacency_list)
        communities.add(community)
        for level in sorted(level_node_map.map.keys(), reverse=True):
            for _, node in level_node_map.map[level].items():
                total_shortest_path = sum([parent.number_of_shortest_paths for parent in node.parent])
                for parent in node.parent:
                    credit = node.credit * parent.number_of_shortest_paths / total_shortest_path
                    parent.credit += credit
                    ec.update((node.id, parent.id), credit)
    
    ec.divide_by_2()   
    ec = sorted([(edge, credit)for edge, credit in ec.map.items()], key=lambda x: x[0][0])
    ec = sorted(ec, key=lambda x: -x[1])
    return ec, communities

In [9]:
def save_edge_credits(output_path, output):
    with open(output_path, 'wt') as f:
        for line in output:
            f.write(line)
    return

def save_output(output_path, output):
    output = sorted(sorted(list(output), key=lambda x: x[0]), key=lambda x: len(x))
    output = ["'"+"', '".join(x)+"'\n" for x in output]
    file = open(output_path, 'wt')
    for line in output:
        file.write(line)
        
    file.close()
    return

st = time.time()
input_path = './data/power_input.txt'
edges = sc.textFile(input_path).map(lambda x: tuple(sorted(x.split()))).collect()
vertices = sc.textFile(input_path).flatMap(lambda x: x.split()).distinct().collect()
original_number_of_edges = len(edges)
original_adjacency_list = create_adjacency_list(edges, vertices)
last_modularity = -3
stopper_count = 0
max_modularity = -sys.maxsize
communities_with_max_modularity = None
while len(edges) != 0:
    if(stopper_count > 15):
        break
    adjacency_list = create_adjacency_list(edges, vertices)
    edge_credits, communities = calculate_betweeness_and_communities(adjacency_list, vertices)
    modularity = calculate_modularity(original_adjacency_list, communities, original_number_of_edges)
    
    if(modularity > max_modularity):
#         print(len(communities), modularity)
        max_modularity = modularity
        communities_with_max_modularity = communities
        stopper_count = 0
    
    if(modularity < last_modularity):
        stopper_count += 1
        
    if(original_number_of_edges == len(edges)):
        output = ["{}, {}\n".format(edge, credit) for edge, credit in edge_credits]
        save_edge_credits('./data/output2.1.txt', output)
    
    last_modularity = modularity
    edges.remove(edge_credits[0][0])

save_output("./data/output2.2.txt", communities_with_max_modularity)
print(time.time()-st)

1 0.0009773572308554238
2 0.22176415264435748
3 0.3062761623515298
4 0.3537134701694212
5 0.37882567285059604
6 0.3852569209927354
7 0.3928985205327187
8 0.4027403159705413
9 0.4094596850040664
10 0.4125626190859069
11 0.4156874820305154
12 0.4169304221549739
15 0.4169779346909744
16 0.417058340521128
17 0.41785143439128
18 0.4189722429328157
19 0.4193048306848147
118.42400121688843


In [113]:
lm, c = BFS('e', original_adjacency_list)

In [114]:
lm.map

{0: {'e': e, 0, 1},
 1: {'z': z, 1, 1, 'd': d, 1, 1, 'f': f, 1, 1},
 2: {'xz': xz, 2, 1, 'g': g, 2, 2, 'b': b, 2, 1},
 3: {'a': a, 3, 1, 'c': c, 3, 1}}

In [75]:
lm, c = BFS('10', original_adjacency_list)

{'0', '1'}


In [71]:
'0' in c

False

In [84]:
# for v in vertices:
#     lm, comm = BFS(v, original_adjacency_list)
#     print(v, set(vertices) == comm)

In [25]:
edge_map, c = calculate_betweeness_and_communities(original_adjacency_list)

In [38]:
set(c[0]) -set(c[4])

{'4', '9'}

In [12]:
[tuple(original_adjacency_list.keys())] == c

False

In [13]:
len(c)

654

In [11]:
calculate_modularity(original_adjacency_list, [tuple(original_adjacency_list.keys())], original_number_of_edges)

0.6517053954329488

In [ ]:
'449', '468', '495', '502', '522', '530', '531', '546', '547', '551', '553', '554', '562'
'589', '592', '597', '598', '602', '604', '610', '611', '623', '624', '627', '635', '640', '641'
'108', '112', '138', '179', '202', '232', '238', '243', '250', '26', '261', '282', '284', '29', '319', '345', '388', '406', '416', '442', '54', '58', '59'
'306', '378', '379', '434', '441', '544', '571', '590', '591', '634', '648', '649', '650', '651', '652', '653', '654', '655', '656', '657', '658', '659', '660', '661', '662'
'587', '588', '595', '600', '601', '603', '605', '607', '608', '609', '612', '616', '617', '625', '626', '628', '629', '630', '631', '632', '633', '636', '637', '638', '639'
'327', '344', '458', '463', '465', '471', '476', '485', '488', '494', '498', '499', '503', '506', '514', '516', '519', '535', '540', '543', '548', '549', '556', '575', '576', '579', '585'
'323', '393', '408', '411', '413', '456', '518', '526', '564', '566', '593', '594', '596', '599', '606', '613', '614', '615', '618', '619', '620', '621', '622', '642', '643', '644', '645', '646', '647'
'100', '113', '115', '121', '125', '134', '156', '174', '184', '194', '204', '206', '217', '252', '258', '265', '275', '278', '281', '283', '285', '290', '36', '37', '392', '55', '63', '68', '74', '82'
'137', '143', '15', '165', '182', '187', '208', '209', '210', '215', '234', '235', '24', '242', '255', '268', '295', '304', '346', '355', '358', '404', '45', '47', '65', '7', '76', '79', '86', '99'
'1', '126', '13', '136', '139', '148', '166', '167', '168', '169', '173', '196', '220', '23', '246', '247', '259', '260', '269', '286', '292', '3', '34', '396', '4', '48', '51', '85', '89', '90', '94', '95'
'107', '110', '12', '131', '133', '135', '14', '140', '158', '16', '160', '162', '163', '18', '189', '192', '193', '226', '228', '245', '25', '251', '254', '276', '277', '28', '280', '289', '33', '46', '57', '60', '80', '81'
'114', '118', '171', '180', '181', '185', '186', '195', '207', '240', '279', '297', '302', '325', '329', '331', '337', '347', '372', '383', '399', '402', '422', '429', '435', '445', '446', '447', '448', '50', '77', '84', '87', '92'
'451', '455', '457', '461', '462', '464', '470', '472', '475', '477', '479', '480', '481', '484', '491', '497', '500', '507', '508', '511', '513', '523', '528', '529', '532', '536', '537', '539', '552', '557', '558', '560', '561', '565', '570', '574', '582', '583', '584'
'101', '104', '105', '116', '117', '119', '120', '129', '130', '142', '144', '159', '161', '175', '197', '199', '200', '203', '205', '211', '221', '233', '244', '253', '267', '291', '296', '324', '357', '38', '384', '389', '39', '390', '391', '417', '66', '67', '71', '72', '73', '78', '8', '88'
'102', '123', '146', '150', '170', '188', '198', '21', '214', '22', '224', '227', '229', '241', '256', '262', '27', '270', '273', '288', '309', '31', '321', '326', '328', '332', '341', '342', '367', '368', '41', '418', '437', '438', '439', '44', '440', '61', '69', '70', '75', '96', '97', '98'
'103', '127', '164', '225', '257', '299', '300', '301', '307', '308', '314', '315', '317', '320', '322', '330', '336', '338', '339', '340', '343', '349', '350', '352', '371', '373', '374', '375', '377', '381', '382', '386', '397', '398', '400', '401', '405', '419', '421', '430', '431', '432', '433', '443', '444', '64'
'450', '452', '453', '454', '459', '460', '466', '467', '469', '473', '474', '478', '482', '483', '486', '487', '489', '490', '492', '493', '496', '501', '504', '505', '509', '510', '512', '515', '517', '520', '521', '524', '525', '527', '533', '534', '538', '541', '542', '545', '550', '555', '559', '563', '567', '568', '569', '572', '573', '577', '578', '580', '581', '586'
'10', '106', '109', '11', '111', '122', '124', '128', '132', '141', '149', '151', '152', '154', '155', '157', '17', '176', '183', '19', '190', '191', '20', '201', '212', '213', '216', '219', '223', '230', '231', '236', '239', '264', '266', '271', '287', '293', '294', '30', '303', '32', '353', '356', '40', '403', '42', '424', '43', '49', '5', '56', '6', '62', '83', '93'
'145', '147', '153', '172', '177', '178', '2', '218', '222', '237', '248', '249', '263', '272', '274', '298', '305', '310', '311', '312', '313', '316', '318', '333', '334', '335', '348', '35', '351', '354', '359', '360', '361', '362', '363', '364', '365', '366', '369', '370', '376', '380', '385', '387', '394', '395', '407', '409', '410', '412', '414', '415', '420', '423', '425', '426', '427', '428', '436', '52', '53', '9', '91'
